In [ ]:
import PyPDF2
import os

# Define the path to the PDF file
pdf_path = r'C:\Users\NCS\Downloads\CV-JD-Matching-main\CV-JD-Matching-main\data\INFORMATION-TECHNOLOGY\89413122.pdf'
output_path = 'output_text.txt'

try:
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # Initialize a variable to store all the text from the PDF
        all_text = ""
        
        # Loop through all the pages in the PDF
        for page_number, page in enumerate(reader.pages):
            # Extract text from each page and add it to the all_text variable
            text = page.extract_text()
            if text:  # Check if text extraction was successful
                all_text += text + "\n"  # Add a newline character after each page's text for better readability
            else:
                print(f"Warning: No text found on page {page_number + 1}")

    # Save the extracted text to a text file
    with open(output_path, 'w', encoding='utf-8') as output_file:
        output_file.write(all_text)
    print(f"Text successfully written to {output_path}")

except FileNotFoundError:
    print(f"Error: File not found at {pdf_path}")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import os

nlp = spacy.blank("en")  # load a new spacy model
db = DocBin()

folder_path = r'C:\Users\NCS\Desktop\IR_CVS'  # Path to your folder

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r', encoding='utf-8') as f:
        TRAIN_DATA = json.load(f)

        # Check if 'annotations' is present and is not None
        if 'annotations' in TRAIN_DATA and TRAIN_DATA['annotations']:
            # Filter out None values before iterating
            annotations = [item for item in TRAIN_DATA['annotations'] if item is not None]
            for text, annot in tqdm(annotations):
                doc = nlp.make_doc(text)
                ents = []
                for start, end, label in annot["entities"]:
                    span = doc.char_span(start, end, label=label, alignment_mode="contract")
                    if span is None:
                        print("Skipping entity")
                    else:
                        ents.append(span)
                doc.ents = ents
                db.add(doc)
        else:
            print(f"No valid annotations found in {filename}")

db.to_disk("./training_data_skills.spacy")


In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load(r'C:\Users\NCS\Desktop\model-best')  # Load a model


file_path = r'C:\Users\NCS\Desktop\output_text.txt'

with open(file_path, 'r') as file:
    content = file.read()

doc = nlp(content)

extracted_skills_cv = []
for ent in doc.ents:
    extracted_skills_cv.append(ent.text)
print(extracted_skills_cv)